In [1]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")

D=6;
chi=3;
W=3;
N=4;
tol=1e-6;

Dtrun=W*W*chi*chi;

pow=Int((N-2)/2);

J1=1;
J2=0;
J3=0;
Jchi=0;
Jtrip=0;


In [2]:

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

filenm="LS_D_"*string(D)*"_chi_40.json"
json_dict=read_json_state(filenm);

bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;

U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];

init=Dict([("CTM", []), ("init_type", "PBC")]);
conv_check="singular_value";
@time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,tol,init);

@time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_triangle");
@time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_bond");


"initialize CTM"

"start CTM iterations:"

 18.442902 seconds (38.62 M allocations: 2.422 GiB, 6.80% gc time, 94.50% compilation time)
CTMRG iteration: 1, CTMRG err: 0.81789132387926
CTMRG iteration: 2, CTMRG err: 0.0039131342976477706
CTMRG iteration: 3, CTMRG err: 0.05748937644566509
CTMRG iteration: 4, CTMRG err: 0.020707834731694477
CTMRG iteration: 5, CTMRG err: 0.007014178298795193
CTMRG iteration: 6, CTMRG err: 0.0023919274479110224
CTMRG iteration: 7, CTMRG err: 0.0007949790148085836
CTMRG iteration: 8, CTMRG err: 0.00025453424786677924
CTMRG iteration: 9, CTMRG err: 7.733757654499651e-5
CTMRG iteration: 10, CTMRG err: 2.1927184551576565e-5
CTMRG iteration: 11, CTMRG err: 5.56408672862068e-6
CTMRG iteration: 12, CTMRG err: 1.1218145512652987e-6
CTMRG iteration: 13, CTMRG err: 8.326572710779872e-8
143.339232 seconds (314.73 M allocations: 20.163 GiB, 5.11% gc time, 97.74% compilation time)
134.657558 seconds (327.22 M allocations: 19.635 GiB, 5.19% gc time, 98.03% compilation time)
 10.959712 seconds (28.82 M allocations

In [3]:
display((E_up+E_down)/3)
Tleft=CTM["Tset"][4];
Tright=CTM["Tset"][2];

@tensor O1[:]:=Tleft[-3,1,-1]*U_L[1,-2,-4];
@tensor O2[:]:=Tright[-1,1,-3]*U_R[-4,-2,1];


-0.4238813693708237 + 0.00019119956884959316im

In [4]:
function HV_L_tensor(vl,A,mpo)
    if mpo==[]
        if numind(vl)==3
            @tensor vl[:]:=vl[-1,3,1]*A'[3,-2,2]*A[1,-3,2];
        elseif numind(vl)==2
            @tensor vl[:]:=vl[3,1]*A'[3,-2,2]*A[1,-3,2];
        end
    else
        if numind(vl)==5
            @tensor vl[:]:=vl[-1,7,5,3,1]*A'[7,-2,6]*mpo'[5,6,-3,4]*mpo[3,2,-4,4]*A[1,-5,2];
        elseif numind(vl)==4
            @tensor vl[:]:=vl[7,5,3,1]*A'[7,-1,6]*mpo'[5,6,-2,4]*mpo[3,2,-3,4]*A[1,-4,2];
        end
    end
    return vl
end

function HV_R_tensor(vr,A,mpo)
    if mpo==[]
        if numind(vr)==3
            @tensor vr[:]:=A'[-1,1,2]*A[-2,3,2]*vr[1,3,-3];
        elseif numind(vr)==2
            @tensor vr[:]:=A'[-1,1,2]*A[-2,3,2]*vr[1,3];
        end
    else
        if numind(vr)==5
            @tensor vr[:]:=A'[-1,7,6]*mpo'[-2,6,5,4]*mpo[-3,2,3,4]*A[-4,1,2]*vr[7,5,3,1,-5];
        elseif numind(vr)==4
            @tensor vr[:]:=A'[-1,7,6]*mpo'[-2,6,5,4]*mpo[-3,2,3,4]*A[-4,1,2]*vr[7,5,3,1];
        end
    end
    return vr
end

function left_eigenvector(A,mpo,type)

    if type=="A"
        HVfun1(x)=HV_L_tensor(x,A,[]);
        vl_init = permute(TensorMap(randn, space(A,1), space(A,1)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun1, vl_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    elseif type=="mpo_A"
        HVfun2(x)=HV_L_tensor(x,A,mpo);
        vl_init = permute(TensorMap(randn, space(A,1)*space(mpo,1),space(mpo,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun2, vl_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    end
end

function right_eigenvector(A,mpo,type)

    if type=="A"
        HVfun1(x)=HV_R_tensor(x,A,[]);
        vr_init = permute(TensorMap(randn, space(A,2), space(A,2)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun1, vr_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    elseif type=="mpo_A"
        HVfun2(x)=HV_R_tensor(x,A,mpo);
        vr_init = permute(TensorMap(randn, space(A,2)*space(mpo,3),space(mpo,3)*space(A,2)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun2, vr_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    end
end



function left_eigenvalue(A1,A2,n)
    function HV_L_A1A2(vl,A1,A2)
        if numind(vl)==3
            @tensor vl[:]:=vl[-1,3,1]*A1'[3,-2,2]*A2[1,-3,2];
        elseif numind(vl)==2
            @tensor vl[:]:=vl[3,1]*A1'[3,-2,2]*A2[1,-3,2];
        end
    return vl
    end
    HVA1A2fun(x)=HV_L_A1A2(x,A1,A2);
    vl_init = permute(TensorMap(randn, space(A1,1), space(A2,1)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
    eu,ev=eigsolve(HVA1A2fun, vl_init, 1,:LM,Arnoldi());
    @assert maximum(abs.(eu)) == abs(eu[1])
    eu=eu[1:n];
    return eu
end

left_eigenvalue (generic function with 1 method)

In [5]:
function impo_imps(mpo,A)
    unitary_mpo_A=unitary(fuse(space(mpo,1)⊗space(A,1)), space(mpo,1)⊗space(A,1));
    @tensor mpo_A[:]:=unitary_mpo_A[-1,2,1]*mpo[2,3,4,-3]*A[1,5,3]*unitary_mpo_A'[4,5,-2];
    return mpo_A
end

impo_imps (generic function with 1 method)

In [6]:
function itebd_ite(mpo,A,W,type)
    if type=="A"
        mpo_A=impo_imps(mpo,A);
        vl,_=left_eigenvector(mpo_A,[],"A");
        eu,ev=eig(permute(vl,(1,),(2,)));
        unitary_L=sqrt(eu)*ev';
        unitary_R=ev*pinv(sqrt(eu));
        @tensor AL[:]:=unitary_L[-1,1]*mpo_A[1,2,-3]*unitary_R[2,-2];
    elseif type=="mpo_A"
        vl,_=left_eigenvector(A,mpo,"mpo_A");
        eu,ev=eig(permute(vl,(2,1,),(3,4,)));
        unitary_L=sqrt(eu)*ev';
        unitary_R=ev*pinv(sqrt(eu));
        @tensor AL[:]:=unitary_L[-1,2,1]*mpo[2,3,4,-3]*A[1,5,3]*unitary_R[4,5,-2];
    end

    vr,_=right_eigenvector(AL,[],"A");

    U,S,_=tsvd(permute(vr,(1,),(2,)),trunc=truncdim(W));
    #display(diag(convert(Array,S)))
    @tensor A_trun[:]:=U[1,-1]*AL[1,2,-3]*U'[-2,2];

    return A_trun
end


itebd_ite (generic function with 1 method)

In [7]:
function left_right_normalize(A)
    #left normalize
    vl,_=left_eigenvector(A,[],"A");
    eu,ev=eig(permute(vl,(1,),(2,)));
    unitary_L=sqrt(eu)*ev';
    unitary_R=ev*pinv(sqrt(eu));
    @tensor AL[:]:=unitary_L[-1,1]*A[1,2,-3]*unitary_R[2,-2];
    #transform right fixed point to be diagonal
    vr,eu=right_eigenvector(AL,[],"A");
    U,S,_=tsvd(permute(vr,(1,),(2,)));
    @tensor AL[:]:=U[1,-1]*AL[1,2,-3]*U'[-2,2];
    AL=AL/sqrt(eu);
    return AL
end


left_right_normalize (generic function with 1 method)

In [9]:
function ITEBD_boundary_groundstate(O1,O2,W,A_init,method)
# ITEBD_boundary_groundstate(O1,O2,D,inv_tol):
    #use power method to obtain dominant boundary imps of nonhermitian MPO
    if method=="OO"
        @tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
        U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
        @tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
    end
    #Initial state
    if A_init==[]
        mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
        A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());
    else 
    end
    AL=left_right_normalize(A_init);
    
    for cp=1:30
        AL_old=AL;
        @time if method=="O_O"
            A_trun_intermed=itebd_ite(O1,AL,3*W,"mpo_A");
            A_trun=itebd_ite(O2,A_trun_intermed,W,"mpo_A");
            AL=left_right_normalize(A_trun);
        elseif method=="OO"
            A_trun=itebd_ite(OO,AL,W,"mpo_A");#this "mpo_A" option is slightly faster when bond dimension is large
        end
        AL=left_right_normalize(A_trun);

        # vv,ee=right_eigenvector(AL,[],"A");
        # display(ee)
        # u,s,v=tsvd(permute(vv,(1,),(2,)));
        # display(sort(diag(convert(Array,s)),rev=true))
        
        #dominant eigenvalue of transfer matrix
        E=left_eigenvalue(impo_imps(O2,impo_imps(O1,AL)),AL,1)[1]
        ov=abs(left_eigenvalue(AL,AL_old,1)[1]);
        println("E="*string(E)*", "*"ov="*string(ov))
        if abs(ov-1)<1e-8
            break
        end
    
    end
    return AL,A_init
end

ITEBD_boundary_groundstate (generic function with 1 method)

In [10]:
mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());



In [11]:
#Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"OO");


In [12]:

Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"O_O");

  7.950717 seconds (17.47 M allocations: 932.550 MiB, 3.21% gc time, 98.77% compilation time)
E=0.21465343977357543 - 0.02202736687236164im, ov=0.629279690638117
  0.053282 seconds (430.13 k allocations: 38.151 MiB, 0.86% compilation time)
E=0.21456681066155897 - 0.020250132743813792im, ov=0.9986436579551982
  0.054066 seconds (385.09 k allocations: 32.693 MiB)
E=0.2149661561093026 - 0.019242953474403288im, ov=0.9996516774790308
  0.054739 seconds (387.22 k allocations: 32.835 MiB)
E=0.21533517341526107 - 0.018835317009744928im, ov=0.9998378809148264
  0.053755 seconds (387.22 k allocations: 32.835 MiB)
E=0.21557026216679923 - 0.018700752066631975im, ov=0.9999038772714336
  0.055236 seconds (387.21 k allocations: 32.834 MiB)
E=0.21567930673807406 - 0.01868326466483403im, ov=0.9999353666340145
  0.056434 seconds (391.68 k allocations: 33.242 MiB)
E=0.2156941994278431 - 0.01870825941827603im, ov=0.9999528351409798
  0.058849 seconds (391.68 k allocations: 33.242 MiB)
E=0.2156465405493582

In [13]:
#save initial CTM to compare with other codes
matwrite("itebd_matfile.mat", Dict(
    "O1" => convert(Array,O1),
    "O2" => convert(Array,O2),
    "A_init" => convert(Array,A_init),
    "Ag" => convert(Array,Ag)
); compress = false)



In [14]:
#save data
O1_dict=convert(Dict,O1);
O2_dict=convert(Dict,O2);
Ag_dict=convert(Dict,Ag);
save("itebd_data.jld", "O1_dict", O1_dict,"O2_dict", O2_dict,"Ag_dict", Ag_dict);




In [15]:
#load data
Ag_dict=load("itebd_data.jld")["Ag_dict"];
O1_dict=load("itebd_data.jld")["O1_dict"];
O2_dict=load("itebd_data.jld")["O2_dict"];
Ag=convert(TensorMap,Ag_dict);
O1=convert(TensorMap,O1_dict);
O2=convert(TensorMap,O2_dict);


In [16]:

space_AOA=fuse(space(Ag,1)'⊗space(O2,1)'⊗space(O1,1)⊗ space(Ag,1));
display(space_AOA)
display(dim(space_AOA))
space_AA=fuse(space(Ag,1)'⊗ space(Ag,1));
display(space_AA)
display(dim(space_AA))

Rep[SU₂](0=>9, 1/2=>12, 1=>9, 3/2=>4, 2=>1)

81

Rep[SU₂](0=>2, 1/2=>2, 1=>1)

9

In [17]:
# W=20;chi=20;
# (Rep[SU₂](0=>3, 1/2=>3, 1=>2, 3/2=>1)' ⊗ Rep[SU₂](0=>3, 1/2=>3, 1=>2, 3/2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>2, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()
# Rep[SU₂](0=>4999, 1/2=>8314, 1=>9146, 3/2=>7850, 2=>5503, 5/2=>3190, 3=>1521, 7/2=>584, 4=>173, 9/2=>36, 5=>4)

In [18]:
for s in sectors(space_AOA)
  display(s)
  @show s, dim(space_AOA, s), dim(s)
end


Irrep[SU₂](0)

Irrep[SU₂](1/2)

Irrep[SU₂](1)

Irrep[SU₂](3/2)

Irrep[SU₂](2)

(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](0), 9, 1)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](1/2), 12, 2)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](1), 9, 3)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](3/2), 4, 4)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](2), 1, 5)


In [19]:
AOA_sec=collect(sectors(space_AOA))
AOA_sec[1]
AA_sec=collect(sectors(space_AA))
AA_sec[1]


Irrep[SU₂](0)

In [20]:
@tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
@tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
display(space(OO))


(Rep[SU₂](0=>2, 1/2=>2, 1=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1) ⊗ Rep[SU₂](0=>2, 1/2=>2, 1=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [90]:
# #attention: when the dimension is slightly large, converting dense matrix to tensormap is extremely difficult. It's better to define tensormap from Dict
# using TensorKit
# sp=Rep[SU₂](0=>129, 1/2=>196, 1=>185, 3/2=>124, 2=>61, 5/2=>20, 3=>4); #out of memory error for this dimension
# sp=Rep[SU₂](0=>12, 1/2=>19, 1=>15, 3/2=>14, 2=>61, 5/2=>20, 3=>4); # it takes long time
# tt=TensorMap(randn,sp←sp);
# tt_dense=convert(Array,tt);
# tt_new=TensorMap(tt_dense,sp,sp);
function combine_singlespin_sector_bruteforce(E_set,vL_set,vR_set,spin_set)
    E_set_new=copy(euL_set);
    vL_set_new=copy(evL_set);
    vR_set_new=copy(evR_set);
    spin_set_new=copy(SPIN_eig_set);
    for cc=1:length(E_set)
        E=E_set[cc];
        vL=vL_set[cc];
        vR=vR_set[cc];
        spin=spin_set[cc];
        if length(vL)>0
            single_size=size(convert(Array,vL[1]));
            if length(single_size)==4
                vL_grouped=zeros(Complex{Float64},single_size[1]*length(vL),single_size[2]*single_size[3]*single_size[4]);
                vR_grouped=zeros(Complex{Float64},single_size[2]*single_size[3]*single_size[4],single_size[1]*length(vL));
                E_grouped=zeros(Complex{Float64},single_size[1]*length(vL),single_size[1]*length(vL));
                spin_dim=single_size[1];
                Unitary=unitary(domain(vL[1]),fuse(domain(vL[1])));
                for cs=1:length(E)
                    vL_grouped[spin_dim*(cs-1)+1:spin_dim*cs,:]=convert(Array,vL[cs]*Unitary);
                    vR_grouped[:,spin_dim*(cs-1)+1:spin_dim*cs]=convert(Array,Unitary'*vR[cs]);
                    E_grouped[spin_dim*(cs-1)+1:spin_dim*cs,spin_dim*(cs-1)+1:spin_dim*cs]=E[cs]*Matrix(I, spin_dim,spin_dim);
                end
                space_single_spin=SU₂Space(spin[1]=>length(E));

                vL_grouped=TensorMap(vL_grouped,space_single_spin←fuse(domain(vL[1])));
                vR_grouped=TensorMap(vR_grouped,fuse(domain(vL[1])) ← space_single_spin);
                E_grouped=TensorMap(E_grouped,space_single_spin ← space_single_spin);

                vL_grouped=vL_grouped*Unitary';
                vR_grouped=Unitary * vR_grouped;

                vL_set_new[cc]=vL_grouped;
                vR_set_new[cc]=vR_grouped;
                E_set_new[cc]=E_grouped;
                spin_set_new[cc]=spin_set[cc][1]
                # display(Base.summarysize(vL_grouped))
                # display(space(vL_grouped))
                # display(Base.summarysize(vL))
            elseif length(single_size)==3
            end
        end
    end
    return E_set_new,vL_set_new,vR_set_new,spin_set_new
end


combine_singlespin_sector_bruteforce (generic function with 1 method)

In [82]:
# function combine_singlespin_sector(E_set,vL_set,vR_set,spin_set)
#     E_set_new=copy(E_set);
#     vL_set_new=copy(vL_set);
#     vR_set_new=copy(vR_set);
#     spin_set_new=copy(spin_set);
#     for cc=1:length(E_set)
#         E=E_set[cc];
#         vL=vL_set[cc];
#         vR=vR_set[cc];
#         spin=spin_set[cc];
#         if length(vL)>0

#             sec=Irrep[SU₂](spin[1]);
#             space_single_spin=SU₂Space(spin[1]=>length(E));
#             dim_full=dim(fuse(domain(vL[1])),sec);
#             vL_grouped=zeros(Complex{Float64},length(vL),dim_full);
#             vR_grouped=zeros(Complex{Float64},dim_full,length(vL));
#             E_grouped=zeros(Complex{Float64},length(vL),length(vL));

#             Unitary=unitary(domain(vL[1]),fuse(domain(vL[1])));
#             for cs=1:length(E)
#                 vL_grouped[cs,:]=convert(Dict,vL[cs]*Unitary)[:data][string(sec)];
#                 vR_grouped[:,cs]=convert(Dict,Unitary'*vR[cs])[:data][string(sec)];
#                 E_grouped[cs,cs]=E[cs];
#             end

#             vL_grouped_rand=TensorMap(randn,space_single_spin←fuse(domain(vL[1])));
#             vR_grouped_rand=TensorMap(randn,fuse(domain(vL[1])) ← space_single_spin);
#             E_grouped_rand=TensorMap(randn,space_single_spin ← space_single_spin);

#             vL_grouped_dict=convert(Dict,vL_grouped_rand);
#             vL_grouped_dict[:data][string(sec)]=vL_grouped;
#             vL_grouped=convert(TensorMap,vL_grouped_dict);
#             vL_grouped=vL_grouped*Unitary';

#             vR_grouped_dict=convert(Dict,vR_grouped_rand);
#             vR_grouped_dict[:data][string(sec)]=vR_grouped;
#             vR_grouped=convert(TensorMap,vR_grouped_dict);
#             vR_grouped=Unitary*vR_grouped;

#             E_grouped_dict=convert(Dict,E_grouped_rand);
#             E_grouped_dict[:data][string(sec)]=E_grouped;
#             E_grouped=convert(TensorMap,E_grouped_dict);

#             vL_set_new[cc]=vL_grouped;
#             vR_set_new[cc]=vR_grouped;
#             E_set_new[cc]=E_grouped;
#             spin_set_new[cc]=spin_set_new[cc][1]
#             # display(Base.summarysize(vL_grouped))
#             # display(space(vL_grouped))
#             # display(Base.summarysize(vL))

#         end
#     end
#     return E_set_new,vL_set_new,vR_set_new,spin_set_new
# end



combine_singlespin_sector (generic function with 1 method)

In [108]:
function combine_singlespin_sector(E_set,vL_set,vR_set,spin_set,is_eig)
    E_set_new=copy(E_set);
    vL_set_new=copy(vL_set);
    vR_set_new=copy(vR_set);
    spin_set_new=copy(spin_set);
    for cc=1:length(E_set)
        E=E_set[cc];
        vL=vL_set[cc];
        vR=vR_set[cc];
        spin=spin_set[cc];
        if length(vL)>0

            sec=Irrep[SU₂](spin[1]);
            space_single_spin=SU₂Space(spin[1]=>length(E));
            dim_full=dim(fuse(domain(vL[1])),sec);
            vL_grouped=zeros(Complex{Float64},length(vL),dim_full);
            vR_grouped=zeros(Complex{Float64},dim_full,length(vL));
            E_grouped=zeros(Complex{Float64},length(vL),length(vL));

            Unitary=unitary(domain(vL[1]),fuse(domain(vL[1])));
            for cs=1:length(E)
                vL_grouped[cs,:]=convert(Dict,vL[cs]*Unitary)[:data][string(sec)];
                vR_grouped[:,cs]=convert(Dict,Unitary'*vR[cs])[:data][string(sec)];
                E_grouped[cs,cs]=E[cs];
            end

            if is_eig
                Q=zeros(Complex{Float64},length(E),length(E))
                for ca=1:length(E)
                    for cb=1:length(E)
                        Q[ca,cb]=tr(vL[ca]*vR[cb])
                    end
                end
                #Q=vL_grouped*vR_grouped;#the normalization of this line is inccorect, due to the dimension of spin space
                vL_grouped=pinv(Q)*vL_grouped;
            end

            vL_grouped_rand=TensorMap(randn,space_single_spin←fuse(domain(vL[1])));
            vR_grouped_rand=TensorMap(randn,fuse(domain(vL[1])) ← space_single_spin);
            E_grouped_rand=TensorMap(randn,space_single_spin ← space_single_spin);

            vL_grouped_dict=convert(Dict,vL_grouped_rand);
            vL_grouped_dict[:data][string(sec)]=vL_grouped;
            vL_grouped=convert(TensorMap,vL_grouped_dict);
            vL_grouped=vL_grouped*Unitary';

            vR_grouped_dict=convert(Dict,vR_grouped_rand);
            vR_grouped_dict[:data][string(sec)]=vR_grouped;
            vR_grouped=convert(TensorMap,vR_grouped_dict);
            vR_grouped=Unitary*vR_grouped;

            E_grouped_dict=convert(Dict,E_grouped_rand);
            E_grouped_dict[:data][string(sec)]=E_grouped;
            E_grouped=convert(TensorMap,E_grouped_dict);

            vL_set_new[cc]=vL_grouped;
            vR_set_new[cc]=vR_grouped;
            E_set_new[cc]=E_grouped;
            spin_set_new[cc]=spin_set_new[cc][1]
            # display(Base.summarysize(vL_grouped))
            # display(space(vL_grouped))
            # display(Base.summarysize(vL))

        end
    end
    return E_set_new,vL_set_new,vR_set_new,spin_set_new
end




combine_singlespin_sector (generic function with 2 methods)

In [39]:

function truncate_sectors(n,eu_set,ev_set,ev_set2,spin_set)
    eu_full=eu_set[1]
    dim_full=2*spin_set[1].+1
    for cc=2:length(eu_set)
        eu_full=vcat(eu_full, eu_set[cc])
        dim_full=vcat(dim_full, 2*spin_set[cc].+1)
    end
    dim_full=Int.(round.(dim_full))

    order=sortperm(abs.(eu_full),rev=true);
    eu_full_sorted=eu_full[order];
    dim_full_sorted=dim_full[order];

    total_size=dim_full_sorted[1];
    old_value=eu_full_sorted[1];
    for cc=2:length(eu_full_sorted)
        total_size=total_size+dim_full_sorted[cc];
        new_value=eu_full_sorted[cc];
        if total_size>n
            break;
        end
        old_value=new_value;
    end

    for cc=1:length(eu_set)
        spins=spin_set[cc];
        eu=eu_set[cc];
        ev=ev_set[cc];
        inds=findall(x->abs(x)>=abs(old_value), abs.(eu));
        eu=eu[inds];
        ev=ev[inds];
        spins=spins[inds];
        eu_set[cc]=eu;
        ev_set[cc]=ev;
        spin_set[cc]=spins;
        if ~(ev_set2==[])
            ev2=ev_set2[cc];
            ev2=ev2[inds];
            ev_set2[cc]=ev2;
        end
    end
return eu_set,ev_set,ev_set2,spin_set

end


truncate_sectors (generic function with 1 method)

In [41]:
function FLR_eig(A,O,n,full_space,sector_set)
    SPIN_set=Vector{Any}(undef, length(sector_set));
    euL_set=Vector{Any}(undef, length(sector_set));
    evL_set=Vector{Any}(undef, length(sector_set));
    euR_set=Vector{Any}(undef, length(sector_set));
    evR_set=Vector{Any}(undef, length(sector_set));
    println("FLR_eig: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        println("spin "*string(spin))

        FLR_eig_L(x)=HV_FL(x,A,O);
        vl_init=permute(TensorMap(randn, SU₂Space(spin=>1),space(A',1)*space(O,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            euL,evL,info=eigsolve(FLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            euL,evL,info=eigsolve(FLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-14));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(euL)==sort(abs.(euL), rev=true)
        euL=euL[1:minimum([length(euL),n_eff,dim(full_space,sec)])];
        evL=evL[1:minimum([length(evL),n_eff,dim(full_space,sec)])];
        euL_set[cc]=euL;
        evLL=evL;
        evL=Vector{Any}(undef, length(evLL));
        for cs=1:length(evL)
            evL[cs]=permute(evLL[cs],(1,),(2,3,4,))
        end
        evL_set[cc]=evL;
        SPIN_set[cc]=ones(length(euL))*(dim(sec)-1)/2;
        
        FLR_eig_R(x)=HV_FR(x,A,O);
        vr_init=permute(TensorMap(randn, space(A',2)'*space(O,3)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            euR,evR,info=eigsolve(FLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            euR,evR,info=eigsolve(FLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-14));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(euR)==sort(abs.(euR), rev=true)
        euR=euR[1:minimum([length(euR),n_eff,dim(full_space,sec)])];
        evR=evR[1:minimum([length(evR),n_eff,dim(full_space,sec)])];
        euR_set[cc]=euR;
        evRR=evR;
        evR=Vector{Any}(undef, length(evRR));
        for cs=1:length(evR)
            evR[cs]=permute(evRR[cs],(1,2,3,),(4,))
        end
        evR_set[cc]=evR;

        @assert (norm(abs.(euL)-abs.(euR))/norm(euL))<1e-8
    end
    return euL_set,evL_set,euR_set,evR_set,SPIN_set     
end
        
function HV_FL(vl,A,mpo)
    @tensor vl[:]:=vl[-1,1,3,5]*A'[1,-2,2]*mpo[3,4,-3,2]*A[5,-4,4];
    return vl
end

function HV_FR(vr,A,mpo)
    @tensor vr[:]:=A'[-1,1,2]*mpo[-2,4,3,2]*A[-3,5,4]*vr[1,3,5,-4];
    return vr
end
 
        


HV_FR (generic function with 1 method)

In [42]:
function GLR_eig(A,n,full_space,sector_set)
    spin_set=Vector{Any}(undef, length(sector_set));
    eul_set=Vector{Any}(undef, length(sector_set));
    evl_set=Vector{Any}(undef, length(sector_set));
    eur_set=Vector{Any}(undef, length(sector_set));
    evr_set=Vector{Any}(undef, length(sector_set));
    println("GLR_eig: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        println("spin "*string(spin))
        
        GLR_eig_L(x)=HV_L_tensor(x,A,[]);
        vl_init=permute(TensorMap(randn, SU₂Space(spin=>1),space(A',1)*space(A,1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            eul,evl,info=eigsolve(GLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            eul,evl,info=eigsolve(GLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-14));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(eul)==sort(abs.(eul), rev=true)
        
        eul=eul[1:minimum([length(eul),n_eff,dim(full_space,sec)])];
        evl=evl[1:minimum([length(evl),n_eff,dim(full_space,sec)])];
        eul_set[cc]=eul;
        evll=evl;
        evl=Vector{Any}(undef, length(evll));
        for cs=1:length(evl)
            evl[cs]=permute(evll[cs],(1,),(2,3,))
        end
        evl_set[cc]=evl;
        spin_set[cc]=ones(length(eul))*(dim(sec)-1)/2;

        GLR_eig_R(x)=HV_R_tensor(x,A,[]);
        vr_init=permute(TensorMap(randn, space(A',2)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            eur,evr,info=eigsolve(GLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            eur,evr,info=eigsolve(GLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-14));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(eur)==sort(abs.(eur), rev=true)
        eur=eur[1:minimum([length(eur),n_eff,dim(full_space,sec)])];
        evr=evr[1:minimum([length(evl),n_eff,dim(full_space,sec)])];
        eur_set[cc]=eur;
        evrr=evr;
        evr=Vector{Any}(undef, length(evrr));
        for cs=1:length(evr)
            evr[cs]=permute(evrr[cs],(1,2,),(3,))
        end
        evr_set[cc]=evr;

        @assert (norm(abs.(eul)-abs.(eur))/norm(eul))<1e-8

    end
    return eul_set,evl_set,eur_set,evr_set,spin_set   
end
    
    
    

GLR_eig (generic function with 1 method)

In [43]:
function FLR_svd(A,O,pow,n,full_space,sector_set)
    SPIN_set=Vector{Any}(undef, length(sector_set));
    S_set=Vector{Any}(undef, length(sector_set));
    U_set=Vector{Any}(undef, length(sector_set));
    Vh_set=Vector{Any}(undef, length(sector_set));
    println("FLR_svd: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        println("spin "*string(spin))

        FLR_svd_R(x)=HV_FR_pow(x,A,O,pow);
        FLR_svd_R_conj(x)=HV_FR_conj_pow(x,A,O,pow);
        vl_init=permute(TensorMap(randn, SU₂Space(spin=>1),space(A',1)*space(O,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        vr_init=permute(TensorMap(randn, space(A',2)'*space(O,3)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            S,U,V,info=svdsolve((FLR_svd_R,FLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*2);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of singular values obtained are not enough, use smaller tol")
            S,U,V,info=svdsolve((FLR_svd_R,FLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-14);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(S)==sort(abs.(S), rev=true)
        S=S[1:minimum([length(S),n_eff,dim(full_space,sec)])];
        U=U[1:minimum([length(U),n_eff,dim(full_space,sec)])];
        V=V[1:minimum([length(V),n_eff,dim(full_space,sec)])];

        UU=U;
        U=Vector{Any}(undef, length(UU));
        for cs=1:length(U)
            U[cs]=permute(UU[cs],(1,2,3,),(4,))
        end

        Vh=Vector{Any}(undef, length(V));
        for cs=1:length(Vh)
            vvr=V[cs];
            Vh[cs]=permute(vvr',(4,),(1,2,3,));
        end
        S_set[cc]=S;
        U_set[cc]=U;
        Vh_set[cc]=Vh;
        SPIN_set[cc]=ones(length(S))*(dim(sec)-1)/2;
        
    end
    return S_set,U_set,Vh_set,SPIN_set     
end
            

function HV_FR_pow(vr,A,mpo,pow)
    for cc=1:pow
        vr=HV_FR(vr,A,mpo);
    end
    return vr
end
function HV_FR_conj(vr,A,mpo)
    @tensor vr[:]:=A[1,-1,2]*mpo'[3,4,-2,2]*A'[5,-3,4]*vr[1,3,5,-4];
    return vr
end
function HV_FR_conj_pow(vr,A,mpo,pow)
    for cc=1:pow
        vr=HV_FR_conj(vr,A,mpo);
    end
    return vr
end
    

HV_FR_conj_pow (generic function with 1 method)

In [44]:
function GLR_svd(A,pow,n,full_space,sector_set)
    spin_set=Vector{Any}(undef, length(sector_set));
    s_set=Vector{Any}(undef, length(sector_set));
    u_set=Vector{Any}(undef, length(sector_set));
    vh_set=Vector{Any}(undef, length(sector_set));
    A_noise=TensorMap(randn, codomain(A),domain(A));
    A_noise=A_noise/norm(A_noise)*norm(A)*1e-9;
    println("GLR_svd: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        println("spin "*string(spin))


        vr_init=permute(TensorMap(randn, space(A',2)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            GLR_svd_R(x)=HV_R_pow(x,A,pow,A_noise*0);
            GLR_svd_R_conj(x)=HV_R_conj_pow(x,A,pow,A_noise*0);
            s,u,v,info=svdsolve((GLR_svd_R,GLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*2);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])    
        catch e
            GLR_svd_R(x)=HV_R_pow(x,A,pow,A_noise);
            GLR_svd_R_conj(x)=HV_R_conj_pow(x,A,pow,A_noise);
            println("Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol")
            s,u,v,info=svdsolve((GLR_svd_R,GLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*4, tol=1e-14);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(s)==sort(abs.(s), rev=true)
    
        s=s[1:minimum([length(s),n_eff,dim(full_space,sec)])];
        u=u[1:minimum([length(u),n_eff,dim(full_space,sec)])];
        v=v[1:minimum([length(v),n_eff,dim(full_space,sec)])];

        uu=u;
        u=Vector{Any}(undef, length(u));
        for cs=1:length(u)
            u[cs]=permute(uu[cs],(1,2,),(3,))
        end

        vh=Vector{Any}(undef, length(v));
        for cs=1:length(vh)
            vvr=v[cs];
            vh[cs]=permute(vvr',(3,),(1,2,));
        end
        s_set[cc]=s;
        u_set[cc]=u;
        vh_set[cc]=vh;
        spin_set[cc]=ones(length(s))*(dim(sec)-1)/2;
        
    end
    return s_set,u_set,vh_set,spin_set     
end

function HV_R_pow(vr,A,pow,A_noise)
    A_=A+A_noise;
    for cc=1:pow
        @tensor vr[:]:=A_'[-1,1,2]*A[-2,3,2]*vr[1,3,-3];
    end
    return vr
end

function HV_R_conj_pow(vr,A,pow,A_noise)
    A_=A+A_noise;
    for cc=1:pow
        @tensor vr[:]:=A_[1,-1,2]*A'[3,-2,2]*vr[1,3,-3];
    end
    return vr
end



HV_R_conj_pow (generic function with 1 method)

In [ ]:

function TransfOp_decom(A,O,pow,Dtrun_init,Dtrun_max,trun_tol,method)
    Dstep=50;
    Dtrun=Dtrun_init;
    if method=="eigenvalue_FLR"
        for cs=1:round((Dtrun_max-Dtrun_init)/Dstep)+1
            evL,lambdaL=F_L(A,O,Dtrun);
            evR,lambdaR=F_R(A,O,Dtrun);
    
            eu=abs(lambdaL);
            eu=eu/max(eu); 
            eu=eu.^pow;
            disp(['method=',method,', Dtrun=',num2str(Dtrun),', minimal eigenvalue: ',num2str(min(eu))])
            if min(eu)<trun_tol
                break;
            else
                if Dtrun>(size(A,1)^2)*(size(O,1)^2)
                    disp('Dtrun exceeds matrix size')
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        [eu_sorted,order]=sort(eu,'descend');
        evL=evL(:,order);
        lambdaL=lambdaL(order);
        evR=evR(:,order);
        lambdaR=lambdaR(order);
        
        posit=find(eu_sorted>trun_tol);
        evL=evL(:,posit);
        lambdaL=lambdaL(posit);
        evR=evR(:,posit);
        lambdaR=lambdaR(posit);
        
        return evL,lambdaL,evR,lambdaR

    elseif method=="eigenvalue_GLR"
        for cs=1:round((Dtrun_max-Dtrun_init)/Dstep)+1
            evl,lambdal=G_L(A,A,Dtrun);
            evr,lambdar=G_R(A,A,Dtrun);
    
            eu=abs(lambdal);
            eu=eu/max(eu); 
            eu=eu.^pow;
            disp(['method=',method,', Dtrun=',num2str(Dtrun),', minimal eigenvalue: ',num2str(min(eu))])
            if min(eu)<trun_tol
                break;
            else
                if Dtrun>size(A,1)^2
                    disp('Dtrun exceeds matrix size')
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        [eu_sorted,order]=sort(eu,'descend');
        evl=evl(:,order);
        lambdal=lambdal(order);
        evr=evr(:,order);
        lambdar=lambdar(order);
        
        posit=find(eu_sorted>trun_tol);
        evl=evl(:,posit);
        lambdal=lambdal(posit);
        evr=evr(:,posit);
        lambdar=lambdar(posit);
        
        return evl,lambdal,evr,lambdar
    elseif method=="svd_GLR"
        for cs=1:round((Dtrun_max-Dtrun_init)/Dstep)+1
            [u,s,v]=GLR_svd(A,pow,Dtrun);
            vt=v';
            eu=abs(diag(s));
            eu=eu/max(eu); 
            disp(['method=',method,', Dtrun=',num2str(Dtrun),', minimal singularvalue: ',num2str(min(eu))])
            if min(eu)<trun_tol
                break;
            else
                if Dtrun>(size(A,1)^2)
                    disp('Dtrun exceeds matrix size')
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        [eu_sorted,order]=sort(eu,'descend');
        u=u(:,order);
        s=s(order,order);
        vt=vt(order,:);
    
        posit=find(eu_sorted>trun_tol);
        u=u(:,posit);
        s=s(posit,posit);
        vt=vt(posit,:);
        
        s=diag(s);
        return u,s,vt
    elseif method=="svd_FLR"
        for cs=1:round((Dtrun_max-Dtrun_init)/Dstep)+1
            [U,S,V]=FLR_svd(A,O,pow,Dtrun);
            Vt=V';   
            eu=abs(diag(S));
            eu=eu/max(eu); 
            disp(['method=',method,', Dtrun=',num2str(Dtrun),', minimal singularvalue: ',num2str(min(eu))])
            if min(eu)<trun_tol
                break;
            else
                if Dtrun>(size(A,1)^2)*(size(O,1)^2)
                    disp('Dtrun exceeds matrix size')
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        [eu_sorted,order]=sort(eu,'descend');
        U=U(:,order);
        S=S(order,order);
        Vt=Vt(order,:);
        
        posit=find(eu_sorted>trun_tol);
        U=U(:,posit);
        S=S(posit,posit);
        Vt=Vt(posit,:);
        
        
        S=diag(S);
        
        return U,S,Vt
    end
end
    

    
    


In [87]:

euL_set,evL_set,euR_set,evR_set,SPIN_eig_set=FLR_eig(Ag,OO,Dtrun,space_AOA,AOA_sec);
display(size(euL_set))
eul_set,evl_set,eur_set,evr_set,spin_eig_set=GLR_eig(Ag,Dtrun,space_AA,AA_sec);
display(size(eul_set))
S_set,U_set,Vh_set,SPIN_svd_set=FLR_svd(Ag,OO,pow,Dtrun,space_AOA,AOA_sec);
display(size(S_set))
s_set,u_set,vh_set,spin_svd_set=GLR_svd(Ag,pow,Dtrun,space_AA,AA_sec);
display(size(s_set))


euL_set,evL_set,evR_set,SPIN_eig_set=truncate_sectors(Dtrun,euL_set,evL_set,evR_set,SPIN_eig_set);
display(euL_set)
eul_set,evl_set,evr_set,spin_eig_set=truncate_sectors(Dtrun,eul_set,evl_set,evr_set,spin_eig_set);
display(eul_set)
S_set,U_set,Vh_set,SPIN_svd_set=truncate_sectors(Dtrun,S_set,U_set,Vh_set,SPIN_svd_set);
display(S_set)
s_set,u_set,vh_set,spin_svd_set=truncate_sectors(Dtrun,s_set,u_set,vh_set,spin_svd_set);
display(s_set)

(5,)

(3,)

(5,)

(3,)

5-element Vector{Any}:
 ComplexF64[0.21430674059864965 + 0.01869706811274014im, 0.1223147932051358 + 0.012809174378841668im, 0.06712699526415497 + 0.04084953349296091im, 0.039822640606317836 + 0.06293984641088862im, 0.05934603937316167 - 0.031396672063717476im, 0.041389534564834844 - 0.049080013050491374im, 0.02519241876802619 - 0.004132754419623327im, 0.022172067938786823 + 0.0009716412697776005im, 0.016673705523863665 - 0.0020913450684781586im]
 ComplexF64[0.167022806508933 - 0.014035875169572831im, 0.15481089428752703 + 0.04489175793979814im, 0.07439428015552108 + 0.06223368512054702im, 0.06299349962159698 - 0.05068064912822791im, 0.04314031972521729 + 0.042102672698103756im, 0.039059798836334966 + 0.04400489171012339im, 0.0412690378378704 - 0.032754710925599985im, 0.0403695531485278 - 0.03058754899451058im, 0.027000090786656033 + 0.00675514788517745im, 0.026318017714108605 - 0.008204422788783573im, 0.021238797610134863 + 0.0009563179593934263im, 0.019133281105095937 - 0.00661218530

3-element Vector{Any}:
 ComplexF64[1.0000000000000002 + 2.897095487016407e-17im, 0.605157919231573 + 1.304862491638158e-18im]
 ComplexF64[0.7700001529435955 - 0.13316545899190968im, 0.7700001529435955 + 0.1331654589919097im]
 ComplexF64[0.6146299521228503 + 0.0im]

5-element Vector{Any}:
 [0.27319708356248, 0.1526838557340948, 0.08462934045492332, 0.0797245502896478, 0.06326975233814768, 0.04965213234908977, 0.02466170892664938, 0.01878331293339924, 0.015515859698438468]
 [0.21844753075165022, 0.19636343613141222, 0.10072173660678274, 0.08255394348374756, 0.06796036753884979, 0.06559973525311426, 0.046166431951342464, 0.04310752338969682, 0.024700388623039558, 0.022969697984667477, 0.02008452084482591, 0.018363378343751534]
 [0.16266150382312838, 0.09308055997319777, 0.07717341378647517, 0.06732169635776945, 0.05626788239600043, 0.029209013296394252, 0.02121651301150816, 0.019029685945237546, 0.01620666976481425]
 [0.07410849808752829, 0.04881540553853661, 0.026029059834564244, 0.02411866602882431]
 [0.02182683112691033]

3-element Vector{Any}:
 [1.1802976580809794, 0.5127163602234761]
 [0.7814302744470171, 0.7814302743847984]
 [0.6146299521228495]

FLR_eig: 
spin 0.0
spin 0.5
spin 1.0
spin 1.5
spin 2.0
GLR_eig: 
spin 0.0
spin 0.5
spin 1.0
FLR_svd: 
spin 0.0
spin 0.5
spin 1.0
spin 1.5
spin 2.0
GLR_svd: 
spin 0.0
spin 0.5
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
spin 1.0


┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 2`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142


In [152]:

#check FLR_svd
@tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
M=permute(M,(1,2,3,),(4,5,6,));
M_temp=M*0;
for cc=1:length(S_set)
    S=S_set[cc];
    U=U_set[cc];
    Vh=Vh_set[cc];
    spin=SPIN_svd_set[cc]
    for cs=1:length(S)
        M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
    end
end
display(norm(M-M_temp)/norm(M))

S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
M_temp=M*0;
for cc=1:length(S_set)
    S=S_set_combined[cc];
    U=U_set_combined[cc];
    Vh=Vh_set_combined[cc];
    spin=SPIN_svd_set_combined[cc]
    if S==[]
    else
        M_temp=M_temp+U*S*Vh*(spin*2+1);
    end
end
display(norm(M-M_temp)/norm(M))



#check GLR_svd
@tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
M=permute(M,(1,2,),(3,4,));
M_temp=M*0;
for cc=1:length(s_set)
    s=s_set[cc];
    u=u_set[cc];
    vh=vh_set[cc];
    spin=spin_svd_set[cc]
    for cs=1:length(s)
        M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
    end
end
display(norm(M-M_temp)/norm(M))

s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)


M_temp=M*0;
for cc=1:length(s_set)
    s=s_set_combined[cc];
    vh=vh_set_combined[cc];
    u=u_set_combined[cc];
    spin=spin_svd_set_combined[cc]
    if s==[]
    else
        M_temp=M_temp+u*s*vh*(spin*2+1);
    end
end
display(norm(M-M_temp)/norm(M))


1.4647863520166207e-15

1.469181558374942e-15

3.990397498395882e-6

3.99039749838212e-6

In [153]:
#check FLR_eig
@tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
M=permute(M,(1,2,3,),(4,5,6,));
M_temp=M*0;
for cc=1:length(euL_set)
    euL=euL_set[cc];
    evR=evR_set[cc];
    evL=evL_set[cc];
    spin=SPIN_eig_set[cc]
    for cs=1:length(euL)
        @tensor coe=evR[cs][2,3,4,1]*evL[cs][1,2,3,4];
        M_temp=M_temp+euL[cs]*evR[cs]*evL[cs]*(spin[cs]*2+1)/coe;
    end
end
display(norm(M-M_temp)/norm(M))

euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true)
M_temp=M*0;
for cc=1:length(euR_set)
    euR=euR_set_combined[cc];
    evR=evR_set_combined[cc];
    evL=evL_set_combined[cc];
    spin=SPIN_eig_set_combined[cc]
    if euR==[]
    else
        M_temp=M_temp+evR*euR*evL*(spin*2+1);
    end
end
display(norm(M-M_temp)/norm(M))




#check GLR_eig
@tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
M=permute(M,(1,2,),(3,4,));
eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)

M_temp=M*0;
for cc=1:length(eul_set)
    eur=eur_set_combined[cc];
    evl=evl_set_combined[cc];
    evr=evr_set_combined[cc];
    spin=spin_eig_set_combined[cc]
    if eur==[]
    else
        M_temp=M_temp+evr*eur*evl*(spin*2+1);
    end
end
display(norm(M-M_temp)/norm(M))


5.6796688324491706e-15

3.826753963099069e-15

1.351908387739234e-15

In [124]:
display(eul_set)
display(eur_set)

3-element Vector{Any}:
 ComplexF64[1.0000000000000002 + 2.897095487016407e-17im, 0.605157919231573 + 1.304862491638158e-18im]
 ComplexF64[0.7700001529435955 - 0.13316545899190968im, 0.7700001529435955 + 0.1331654589919097im]
 ComplexF64[0.6146299521228503 + 0.0im]

3-element Vector{Any}:
 ComplexF64[0.9999999999999997 - 1.0703303617385342e-16im, 0.6051579192315732 - 1.637732891428775e-17im]
 ComplexF64[0.7700001529435959 + 0.1331654589919098im, 0.7700001529435953 - 0.13316545899190976im]
 ComplexF64[0.61462995212285 + 0.0im]

In [ ]:
uM,sM,vM=tsvd(M,trunc=truncdim(11));
display(sM)
display(space(uM))
M_reduced=uM*sM*vM-3*u_set[3][1]*s_set[3][1]*vh_set[3][1];

uM,sM,vM=tsvd(M_reduced,trunc=truncdim(8));
display(sM)
display(space(uM))



# #display(sort(diag(convert(Array,sM)),rev=true))
# display(abs(dot(uM,u_set[3][1])/sqrt(3)))
# display(norm(u_set[3][1]))
# display(norm(uM))

# display(abs(dot(vM,vh_set[3][1])/sqrt(3)))
# display(norm(vh_set[3][1]))
# display(norm(vM))

MM=u_set[2][1]*s_set[2][1]*vh_set[2][1]+u_set[2][2]*s_set[2][2]*vh_set[2][2];
uM,sM,vM=tsvd(MM,trunc=truncdim(11));
display(sM)
display(space(uM))

display(abs(dot(u_set[4][1],u_set[4][2])))
display(abs(dot(vh_set[4][1],vh_set[4][2])))

display(norm(uM*sM*vM-4*u_set[4][1]*s_set[4][1]*vh_set[4][1]-4*u_set[4][2]*s_set[4][2]*vh_set[4][2])/norm(uM*sM*vM))


In [127]:
function combine_singlespin_sector_test(E_set,vL_set,vR_set,spin_set,is_eig)
    E_set_new=copy(E_set);
    vL_set_new=copy(vL_set);
    vR_set_new=copy(vR_set);
    spin_set_new=copy(spin_set);
    for cc=1:length(E_set)
        E=E_set[cc];
        vL=vL_set[cc];
        vR=vR_set[cc];
        spin=spin_set[cc];
        if length(vL)>0

            sec=Irrep[SU₂](spin[1]);
            space_single_spin=SU₂Space(spin[1]=>length(E));
            dim_full=dim(fuse(domain(vL[1])),sec);
            vL_grouped=zeros(Complex{Float64},length(vL),dim_full);
            vR_grouped=zeros(Complex{Float64},dim_full,length(vL));
            E_grouped=zeros(Complex{Float64},length(vL),length(vL));

            Unitary=unitary(domain(vL[1]),fuse(domain(vL[1])));
            for cs=1:length(E)
                vL_grouped[cs,:]=convert(Dict,vL[cs]*Unitary)[:data][string(sec)];
                vR_grouped[:,cs]=convert(Dict,Unitary'*vR[cs])[:data][string(sec)];
                E_grouped[cs,cs]=E[cs];
            end

            if is_eig
                Q=zeros(Complex{Float64},length(E),length(E))
                for ca=1:length(E)
                    for cb=1:length(E)
                        Q[ca,cb]=tr(vL[ca]*vR[cb])
                    end
                end
                #Q=vL_grouped*vR_grouped;
                display(Q)
                vL_grouped=pinv(Q)*vL_grouped;
            end

            vL_grouped_rand=TensorMap(randn,space_single_spin←fuse(domain(vL[1])));
            vR_grouped_rand=TensorMap(randn,fuse(domain(vL[1])) ← space_single_spin);
            E_grouped_rand=TensorMap(randn,space_single_spin ← space_single_spin);

            vL_grouped_dict=convert(Dict,vL_grouped_rand);
            vL_grouped_dict[:data][string(sec)]=vL_grouped;
            vL_grouped=convert(TensorMap,vL_grouped_dict);
            vL_grouped=vL_grouped*Unitary';

            vR_grouped_dict=convert(Dict,vR_grouped_rand);
            vR_grouped_dict[:data][string(sec)]=vR_grouped;
            vR_grouped=convert(TensorMap,vR_grouped_dict);
            vR_grouped=Unitary*vR_grouped;

            E_grouped_dict=convert(Dict,E_grouped_rand);
            E_grouped_dict[:data][string(sec)]=E_grouped;
            E_grouped=convert(TensorMap,E_grouped_dict);

            vL_set_new[cc]=vL_grouped;
            vR_set_new[cc]=vR_grouped;
            E_set_new[cc]=E_grouped;
            spin_set_new[cc]=spin_set_new[cc][1]
            # display(Base.summarysize(vL_grouped))
            # display(space(vL_grouped))
            # display(Base.summarysize(vL))

        end
    end
    return E_set_new,vL_set_new,vR_set_new,spin_set_new
end

euL_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector_test(euL_set,evL_set,evR_set,SPIN_eig_set,true)



9×9 Matrix{ComplexF64}:
     0.127486-0.578089im     …   5.63785e-17-2.13371e-16im
  9.31763e-16-6.18429e-16im      3.40006e-16-2.01228e-16im
  4.76182e-16-7.45931e-17im      1.38778e-17-1.04083e-16im
  1.38778e-17+1.96024e-16im     -3.81639e-17-1.66533e-16im
 -8.50015e-17-3.99854e-16im      1.85615e-16-3.46945e-17im
  -2.7452e-16+4.09395e-16im  …  -1.27068e-16+2.15106e-16im
 -2.28983e-16+2.77556e-17im      1.73472e-15-4.16334e-17im
  1.09938e-16+8.67362e-18im      4.57154e-16+7.70217e-16im
 -1.17094e-17-6.44016e-17im         0.632157-0.45563im

12×12 Matrix{ComplexF64}:
    -0.466396-0.559855im     …    -6.245e-17+1.04083e-16im
 -1.97758e-15-1.66533e-16im     -1.68268e-16-3.81639e-17im
  5.27356e-16+3.88578e-16im      1.71738e-16+3.46945e-17im
   -3.747e-16-9.85323e-16im     -1.49186e-16+2.08167e-16im
 -5.59448e-17+2.48716e-16im     -8.43509e-17+2.52185e-16im
  3.60822e-16-7.56339e-16im  …   3.00107e-16-5.0307e-17im
 -3.72966e-17-2.77556e-16im      8.67362e-17-2.498e-16im
 -3.82073e-16+3.46945e-17im      1.78053e-16-4.85723e-16im
 -2.48933e-16-5.58581e-16im     -4.97866e-16-4.48426e-16im
  4.68809e-16+1.31839e-16im     -4.19803e-16+1.13733e-15im
  1.75207e-16+2.22045e-16im  …   3.40006e-16-8.08381e-16im
 -1.38778e-16+8.67362e-17im         0.891384+0.165369im

9×9 Matrix{ComplexF64}:
     0.528108+0.333845im     …    -6.245e-17-1.56125e-16im
 -1.80585e-15-1.18135e-15im     -1.14492e-16-8.32667e-17im
 -2.60209e-16+1.66533e-16im      1.61329e-16-1.06035e-16im
  2.60209e-17-6.79144e-16im      1.30104e-16-8.06646e-17im
 -4.78784e-16+2.05565e-16im      -3.1225e-17+1.30104e-16im
 -5.46438e-17-3.35669e-16im  …   5.25621e-16+5.66604e-16im
  2.62811e-16+1.69136e-17im     -4.31946e-16-5.67255e-16im
  1.43115e-17+4.81386e-17im     -4.51787e-16-2.40693e-17im
  7.02563e-17-6.245e-17im           0.227546-0.928716im

4×4 Matrix{ComplexF64}:
    -0.836908+0.132433im     …  -3.81639e-16+1.249e-16im
 -2.35922e-16+2.08167e-17im      9.22873e-16+9.02056e-17im
  9.99201e-16+6.66134e-16im      4.11997e-17-1.82146e-16im
 -2.77556e-17-4.996e-16im          -0.408488+0.632573im

1×1 Matrix{ComplexF64}:
 -1.0 + 0.0im

(Any[TensorMap(ProductSpace(Rep[SU₂](0=>9)) ← ProductSpace(Rep[SU₂](0=>9))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,), ()):
 0.21430674059864965 + 0.01869706811274014im  …                   0.0 + 0.0im
                 0.0 + 0.0im                                      0.0 + 0.0im
                 0.0 + 0.0im                                      0.0 + 0.0im
                 0.0 + 0.0im                                      0.0 + 0.0im
                 0.0 + 0.0im                                      0.0 + 0.0im
                 0.0 + 0.0im                  …                   0.0 + 0.0im
                 0.0 + 0.0im                                      0.0 + 0.0im
                 0.0 + 0.0im                                      0.0 + 0.0im
                 0.0 + 0.0im                     0.016673705523863665 - 0.0020913450684781586im
, TensorMap(ProductSpace(Rep[SU₂](1/2=>12)) ← ProductSpace(Rep[SU₂](1/2=>12))):
* Data